### 산탄데르 고객 만족 예측
#### 데이터 전처리

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 데이터 로딩 => DataFrame변환

cust_df = pd.read_csv(
    './data/santander/train.csv'
    , encoding='latin-1'
)
cust_df.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [3]:
cust_df.shape
# roe : 76000 , col : 370, label+col : 371

(76020, 371)

In [4]:
cust_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB


In [5]:
# 레이블(TARGET)의 분포 : 균형, 불균형 확인
cust_df['TARGET'].value_counts()

0    73012
1     3008
Name: TARGET, dtype: int64

In [6]:
# 불만족 레이블의 비율 출력
# 불만족 개수 / 전체 개수 : 비율
unsat_cnt = cust_df[cust_df['TARGET']==1].TARGET.count()
total_cnt = cust_df.TARGET.count()
print(unsat_cnt/total_cnt)

0.0395685345961589


In [7]:
cust_df.describe()
# var3 변수 이상치 확인

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,75964.050723,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,...,7.935824,1.365146,12.215580,8.784074,31.505324,1.858575,76.026165,56.614351,1.172358e+05,0.039569
std,43781.947379,39033.462364,12.956486,1614.757313,339.315831,546.266294,93.155749,153.737066,30.604864,36.513513,...,455.887218,113.959637,783.207399,538.439211,2013.125393,147.786584,4040.337842,2852.579397,1.826646e+05,0.194945
min,1.000000,-999999.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.163750e+03,0.000000
25%,38104.750000,2.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.787061e+04,0.000000
50%,76043.000000,2.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.064092e+05,0.000000
75%,113748.750000,2.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.187563e+05,0.000000
max,151838.000000,238.000000,105.000000,210000.000000,12888.030000,21024.810000,8237.820000,11073.570000,6600.000000,6600.000000,...,50003.880000,20385.720000,138831.630000,91778.730000,438329.220000,24650.010000,681462.900000,397884.300000,2.203474e+07,1.000000


In [8]:
# var3 column 확인
# cust_df.var3 == cust_df['var3']
cust_df.var3.value_counts()

 2         74165
 8           138
-999999      116
 9           110
 3           108
           ...  
 177           1
 87            1
 151           1
 215           1
 191           1
Name: var3, Length: 208, dtype: int64

In [9]:
# -999999 -> 2로 변경
cust_df.var3.replace(
      -999999
    , 2
    , inplace=True
)

In [10]:
cust_df.var3.value_counts()

2      74281
8        138
9        110
3        108
1        105
       ...  
177        1
87         1
151        1
215        1
191        1
Name: var3, Length: 207, dtype: int64

In [11]:
# 불필요한 컬럼 삭제 => ID
cust_df.drop(
      'ID' # 삭제할 컬럼/로우
    , axis=1 # 컬럼 지정
    , inplace=True
)

In [12]:
# 학습 테스트 데이터와 답(371컬럼, -1)을 분리
X_features = cust_df.iloc[:,:-1]
y_labels = cust_df.iloc[:,-1]
cust_df.shape, X_features.shape, y_labels.shape

((76020, 370), (76020, 369), (76020,))

In [13]:
# 학습 테스트 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=\
train_test_split(
      X_features
    , y_labels
    , test_size=0.2
    , random_state=0
)

In [14]:
# XGBoost 사용
# n_estimators = 500, early = 100
# 평가지표(eval_metric) : auc
# eval_set=[(X_train, y_train),(X_test, y_test)]
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [28]:
# 모델 객체 생성
xgb_clf = XGBClassifier(
      n_estimators= 500 # 트리 갯수
    , random_state=156
)

In [29]:
# 학습
xgb_clf.fit(
      X_train # 학습 데이터
    , y_train # 학습 답
    , early_stopping_rounds=100
    , eval_metric='auc'
    , eval_set=[(X_train, y_train),(X_test, y_test)]
)

[0]	validation_0-auc:0.799928	validation_1-auc:0.803548
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.802222	validation_1-auc:0.805222
[2]	validation_0-auc:0.80819	validation_1-auc:0.813162
[3]	validation_0-auc:0.8127	validation_1-auc:0.813243
[4]	validation_0-auc:0.81648	validation_1-auc:0.816979
[5]	validation_0-auc:0.816018	validation_1-auc:0.816629
[6]	validation_0-auc:0.816474	validation_1-auc:0.817776
[7]	validation_0-auc:0.818148	validation_1-auc:0.818464
[8]	validation_0-auc:0.81806	validation_1-auc:0.818295
[9]	validation_0-auc:0.817039	validation_1-auc:0.818087
[10]	validation_0-auc:0.818318	validation_1-auc:0.818749
[11]	validation_0-auc:0.818711	validation_1-auc:0.818521
[12]	validation_0-auc:0.818673	validation_1-auc:0.818516
[13]	validation_0-auc:0.819156	validation_1-auc:0.818998
[14]	validation_0-auc:0.819847	validation_1-auc:0.81999
[15

[142]	validation_0-auc:0.861553	validation_1-auc:0.841335
[143]	validation_0-auc:0.861682	validation_1-auc:0.841346
[144]	validation_0-auc:0.86169	validation_1-auc:0.841403
[145]	validation_0-auc:0.861852	validation_1-auc:0.841299
[146]	validation_0-auc:0.861898	validation_1-auc:0.841301
[147]	validation_0-auc:0.861998	validation_1-auc:0.841289
[148]	validation_0-auc:0.862068	validation_1-auc:0.84135
[149]	validation_0-auc:0.862132	validation_1-auc:0.841444
[150]	validation_0-auc:0.862236	validation_1-auc:0.841409
[151]	validation_0-auc:0.862314	validation_1-auc:0.841459
[152]	validation_0-auc:0.862584	validation_1-auc:0.841456
[153]	validation_0-auc:0.862843	validation_1-auc:0.841483
[154]	validation_0-auc:0.863033	validation_1-auc:0.841493
[155]	validation_0-auc:0.863132	validation_1-auc:0.841534
[156]	validation_0-auc:0.863423	validation_1-auc:0.841728
[157]	validation_0-auc:0.863578	validation_1-auc:0.841712
[158]	validation_0-auc:0.863872	validation_1-auc:0.841677
[159]	validation

XGBClassifier(n_estimators=500, random_state=156)

In [30]:
# AUC 값 : 예측확률 => predict_proba()[:,1]
pred_proba = xgb_clf.predict_proba(X_test)[:,1]

# AUC 값
xgb_auc_score = roc_auc_score(
      y_test # 테스트 데이터의 원래 답
    , pred_proba
    , average='macro'
)

print('AUC : {0:.4f}'.format(xgb_auc_score))

AUC : 0.8419


In [32]:
# 하이퍼 파리미터 튜닝
# GridSearchCV
# 객체 생성
xgb_clf = XGBClassifier(
      n_estimators=100
)
# 하이퍼 파라미터 지정
params = {
      'max_depth' : [5,7] # 과적합 방지
    , 'min_child_weight' :  [1,3] # 과적합 방지
    , 'colsample_bytree' : [0.5,0.75] # 과적합 방지    
}

In [33]:
#GridSearchCV 객체 생성
from sklearn.model_selection import GridSearchCV
gridcv = GridSearchCV(
      xgb_clf           # 서치할 모델 객체 지정
    , param_grid=params # 파라미터 지정
)

In [35]:
import time
start_time = time.time()

gridcv.fit(
      X_train
    , y_train
    , early_stopping_rounds=30
    , eval_metric='auc'
    , eval_set=[(X_train, y_train),(X_test, y_test)]
)

end_time = time.time()
print('학습시간 : ', end_time-start_time)

[0]	validation_0-auc:0.716371	validation_1-auc:0.721224
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 30 rounds.
[1]	validation_0-auc:0.808233	validation_1-auc:0.811497
[2]	validation_0-auc:0.804883	validation_1-auc:0.804536
[3]	validation_0-auc:0.815261	validation_1-auc:0.81637
[4]	validation_0-auc:0.825247	validation_1-auc:0.824507
[5]	validation_0-auc:0.819668	validation_1-auc:0.820074
[6]	validation_0-auc:0.818989	validation_1-auc:0.819519
[7]	validation_0-auc:0.824907	validation_1-auc:0.825344
[8]	validation_0-auc:0.828985	validation_1-auc:0.828536
[9]	validation_0-auc:0.827021	validation_1-auc:0.825688
[10]	validation_0-auc:0.83165	validation_1-auc:0.828585
[11]	validation_0-auc:0.834058	validation_1-auc:0.831318
[12]	validation_0-auc:0.836334	validation_1-auc:0.832347
[13]	validation_0-auc:0.835441	validation_1-auc:0.831432
[14]	validation_0-auc:0.834861	validation_1-auc:0.830882


[47]	validation_0-auc:0.862967	validation_1-auc:0.842593
[48]	validation_0-auc:0.863551	validation_1-auc:0.84261
[49]	validation_0-auc:0.864042	validation_1-auc:0.842599
[50]	validation_0-auc:0.864366	validation_1-auc:0.842434
[51]	validation_0-auc:0.865353	validation_1-auc:0.843195
[52]	validation_0-auc:0.866013	validation_1-auc:0.843838
[53]	validation_0-auc:0.866414	validation_1-auc:0.84388
[54]	validation_0-auc:0.86717	validation_1-auc:0.844016
[55]	validation_0-auc:0.867641	validation_1-auc:0.8437
[56]	validation_0-auc:0.868209	validation_1-auc:0.844056
[57]	validation_0-auc:0.868522	validation_1-auc:0.844175
[58]	validation_0-auc:0.868781	validation_1-auc:0.844342
[59]	validation_0-auc:0.869079	validation_1-auc:0.844691
[60]	validation_0-auc:0.869423	validation_1-auc:0.844647
[61]	validation_0-auc:0.869812	validation_1-auc:0.844665
[62]	validation_0-auc:0.870011	validation_1-auc:0.844629
[63]	validation_0-auc:0.870417	validation_1-auc:0.844743
[64]	validation_0-auc:0.870882	valid

[1]	validation_0-auc:0.809291	validation_1-auc:0.810292
[2]	validation_0-auc:0.801509	validation_1-auc:0.805559
[3]	validation_0-auc:0.811599	validation_1-auc:0.814615
[4]	validation_0-auc:0.823122	validation_1-auc:0.824642
[5]	validation_0-auc:0.820774	validation_1-auc:0.820123
[6]	validation_0-auc:0.819431	validation_1-auc:0.819198
[7]	validation_0-auc:0.82599	validation_1-auc:0.825043
[8]	validation_0-auc:0.830975	validation_1-auc:0.829029
[9]	validation_0-auc:0.829664	validation_1-auc:0.827526
[10]	validation_0-auc:0.831341	validation_1-auc:0.829234
[11]	validation_0-auc:0.833655	validation_1-auc:0.83098
[12]	validation_0-auc:0.836167	validation_1-auc:0.832389
[13]	validation_0-auc:0.835919	validation_1-auc:0.830942
[14]	validation_0-auc:0.835294	validation_1-auc:0.831207
[15]	validation_0-auc:0.838006	validation_1-auc:0.832957
[16]	validation_0-auc:0.840265	validation_1-auc:0.834066
[17]	validation_0-auc:0.843274	validation_1-auc:0.837085
[18]	validation_0-auc:0.844687	validation_

[49]	validation_0-auc:0.865152	validation_1-auc:0.840495
[50]	validation_0-auc:0.865554	validation_1-auc:0.840383
[51]	validation_0-auc:0.866651	validation_1-auc:0.841277
[52]	validation_0-auc:0.867289	validation_1-auc:0.841679
[53]	validation_0-auc:0.86781	validation_1-auc:0.842134
[54]	validation_0-auc:0.868175	validation_1-auc:0.842503
[55]	validation_0-auc:0.868746	validation_1-auc:0.842126
[56]	validation_0-auc:0.869264	validation_1-auc:0.842363
[57]	validation_0-auc:0.869795	validation_1-auc:0.842344
[58]	validation_0-auc:0.870406	validation_1-auc:0.842145
[59]	validation_0-auc:0.870529	validation_1-auc:0.842306
[60]	validation_0-auc:0.870793	validation_1-auc:0.842345
[61]	validation_0-auc:0.871081	validation_1-auc:0.842398
[62]	validation_0-auc:0.871391	validation_1-auc:0.84243
[63]	validation_0-auc:0.871823	validation_1-auc:0.842369
[64]	validation_0-auc:0.872147	validation_1-auc:0.842177
[65]	validation_0-auc:0.872387	validation_1-auc:0.842151
[66]	validation_0-auc:0.872716	va

[91]	validation_0-auc:0.873633	validation_1-auc:0.841873
[92]	validation_0-auc:0.873647	validation_1-auc:0.841868
Stopping. Best iteration:
[62]	validation_0-auc:0.867211	validation_1-auc:0.842672

[0]	validation_0-auc:0.720599	validation_1-auc:0.727176
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 30 rounds.
[1]	validation_0-auc:0.813587	validation_1-auc:0.811508
[2]	validation_0-auc:0.809088	validation_1-auc:0.810017
[3]	validation_0-auc:0.816841	validation_1-auc:0.817096
[4]	validation_0-auc:0.825512	validation_1-auc:0.82418
[5]	validation_0-auc:0.819472	validation_1-auc:0.819553
[6]	validation_0-auc:0.818305	validation_1-auc:0.817745
[7]	validation_0-auc:0.824445	validation_1-auc:0.821165
[8]	validation_0-auc:0.829549	validation_1-auc:0.824451
[9]	validation_0-auc:0.827249	validation_1-auc:0.822791
[10]	validation_0-auc:0.831955	validation_1-auc:0.826125
[11]	validation_0-auc:0.83463

...

[38]	validation_0-auc:0.881984	validation_1-auc:0.842147
[39]	validation_0-auc:0.882994	validation_1-auc:0.841905
[40]	validation_0-auc:0.88375	validation_1-auc:0.842009
[41]	validation_0-auc:0.884477	validation_1-auc:0.842543
[42]	validation_0-auc:0.885354	validation_1-auc:0.842277
[43]	validation_0-auc:0.886504	validation_1-auc:0.841963
[44]	validation_0-auc:0.887218	validation_1-auc:0.841897
[45]	validation_0-auc:0.888024	validation_1-auc:0.842132
[46]	validation_0-auc:0.88871	validation_1-auc:0.842191
[47]	validation_0-auc:0.889249	validation_1-auc:0.841935
[48]	validation_0-auc:0.889782	validation_1-auc:0.841548
[49]	validation_0-auc:0.890499	validation_1-auc:0.841694
[50]	validation_0-auc:0.891322	validation_1-auc:0.841738
[51]	validation_0-auc:0.891809	validation_1-auc:0.841493
[52]	validation_0-auc:0.89246	validation_1-auc:0.84135
[53]	validation_0-auc:0.892825	validation_1-auc:0.841669
[54]	validation_0-auc:0.89329	validation_1-auc:0.841769
[55]	validation_0-auc:0.893402	valid

[57]	validation_0-auc:0.893694	validation_1-auc:0.843455
[58]	validation_0-auc:0.894555	validation_1-auc:0.843582
[59]	validation_0-auc:0.894706	validation_1-auc:0.843591
[60]	validation_0-auc:0.894899	validation_1-auc:0.843354
[61]	validation_0-auc:0.895218	validation_1-auc:0.84345
Stopping. Best iteration:
[31]	validation_0-auc:0.875223	validation_1-auc:0.84504

[0]	validation_0-auc:0.824247	validation_1-auc:0.817042
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 30 rounds.
[1]	validation_0-auc:0.828268	validation_1-auc:0.820775
[2]	validation_0-auc:0.83099	validation_1-auc:0.821871
[3]	validation_0-auc:0.835811	validation_1-auc:0.824768
[4]	validation_0-auc:0.836693	validation_1-auc:0.829421
[5]	validation_0-auc:0.843953	validation_1-auc:0.831358
[6]	validation_0-auc:0.844795	validation_1-auc:0.831664
[7]	validation_0-auc:0.846039	validation_1-auc:0.831437
[8]	validation_0-auc:0.84671	

Will train until validation_1-auc hasn't improved in 30 rounds.
[1]	validation_0-auc:0.822797	validation_1-auc:0.817858
[2]	validation_0-auc:0.830112	validation_1-auc:0.822657
[3]	validation_0-auc:0.836008	validation_1-auc:0.82756
[4]	validation_0-auc:0.837619	validation_1-auc:0.828466
[5]	validation_0-auc:0.839996	validation_1-auc:0.82965
[6]	validation_0-auc:0.842981	validation_1-auc:0.830086
[7]	validation_0-auc:0.84369	validation_1-auc:0.831006
[8]	validation_0-auc:0.845052	validation_1-auc:0.831272
[9]	validation_0-auc:0.847985	validation_1-auc:0.834577
[10]	validation_0-auc:0.849627	validation_1-auc:0.834416
[11]	validation_0-auc:0.849852	validation_1-auc:0.834503
[12]	validation_0-auc:0.850294	validation_1-auc:0.834119
[13]	validation_0-auc:0.852049	validation_1-auc:0.836065
[14]	validation_0-auc:0.853922	validation_1-auc:0.836222
[15]	validation_0-auc:0.855645	validation_1-auc:0.837817
[16]	validation_0-auc:0.857736	validation_1-auc:0.838048
[17]	validation_0-auc:0.859355	valid

[9]	validation_0-auc:0.846892	validation_1-auc:0.831069
[10]	validation_0-auc:0.848336	validation_1-auc:0.832402
[11]	validation_0-auc:0.85054	validation_1-auc:0.834388
[12]	validation_0-auc:0.851094	validation_1-auc:0.83519
[13]	validation_0-auc:0.853134	validation_1-auc:0.837028
[14]	validation_0-auc:0.854288	validation_1-auc:0.837004
[15]	validation_0-auc:0.855254	validation_1-auc:0.838322
[16]	validation_0-auc:0.856437	validation_1-auc:0.838067
[17]	validation_0-auc:0.857929	validation_1-auc:0.838312
[18]	validation_0-auc:0.859632	validation_1-auc:0.838508
[19]	validation_0-auc:0.860563	validation_1-auc:0.839477
[20]	validation_0-auc:0.862077	validation_1-auc:0.840735
[21]	validation_0-auc:0.862593	validation_1-auc:0.841008
[22]	validation_0-auc:0.86327	validation_1-auc:0.84181
[23]	validation_0-auc:0.864434	validation_1-auc:0.842597
[24]	validation_0-auc:0.865038	validation_1-auc:0.842642
[25]	validation_0-auc:0.86574	validation_1-auc:0.843322
[26]	validation_0-auc:0.86633	validat

[21]	validation_0-auc:0.865776	validation_1-auc:0.838794
[22]	validation_0-auc:0.86524	validation_1-auc:0.838435
[23]	validation_0-auc:0.867317	validation_1-auc:0.839363
[24]	validation_0-auc:0.868569	validation_1-auc:0.840288
[25]	validation_0-auc:0.869447	validation_1-auc:0.840665
[26]	validation_0-auc:0.86984	validation_1-auc:0.84061
[27]	validation_0-auc:0.872001	validation_1-auc:0.84066
[28]	validation_0-auc:0.873911	validation_1-auc:0.840676
[29]	validation_0-auc:0.874193	validation_1-auc:0.841444
[30]	validation_0-auc:0.875648	validation_1-auc:0.842116
[31]	validation_0-auc:0.87691	validation_1-auc:0.842438
[32]	validation_0-auc:0.877819	validation_1-auc:0.841824
[33]	validation_0-auc:0.878313	validation_1-auc:0.841681
[34]	validation_0-auc:0.878535	validation_1-auc:0.841624
[35]	validation_0-auc:0.881047	validation_1-auc:0.842103
[36]	validation_0-auc:0.883147	validation_1-auc:0.842436
[37]	validation_0-auc:0.883825	validation_1-auc:0.842027
[38]	validation_0-auc:0.884627	valid

- 위 구문에서 xgboost 까지만 실행 후 다른 조건 하 재 분석

In [18]:
# n_estimators=1000, learning_rate=0.02, reg_alpha=0.03
xgb_clf = XGBClassifier(
      n_estimators=1000
    , random_state=156
#     , learning_rate=0.0
    , max_depth=5
    , min_child_weight=1
    , colsample_bytree=0.75
    , reg_alpha=0.03
)

In [19]:
# 학습
xgb_clf.fit(
      X_train
    , y_train
    , early_stopping_rounds=200
    , eval_metric='auc'
    , eval_set= [(X_train, y_train),(X_test, y_test)]
)

[0]	validation_0-auc:0.817284	validation_1-auc:0.811534
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 200 rounds.
[1]	validation_0-auc:0.821595	validation_1-auc:0.81452
[2]	validation_0-auc:0.823928	validation_1-auc:0.815818
[3]	validation_0-auc:0.830391	validation_1-auc:0.825027
[4]	validation_0-auc:0.83374	validation_1-auc:0.826005
[5]	validation_0-auc:0.835684	validation_1-auc:0.828261
[6]	validation_0-auc:0.83606	validation_1-auc:0.826961
[7]	validation_0-auc:0.836359	validation_1-auc:0.827258
[8]	validation_0-auc:0.838213	validation_1-auc:0.828052
[9]	validation_0-auc:0.839832	validation_1-auc:0.828021
[10]	validation_0-auc:0.83922	validation_1-auc:0.829552
[11]	validation_0-auc:0.840079	validation_1-auc:0.830832
[12]	validation_0-auc:0.842225	validation_1-auc:0.831883
[13]	validation_0-auc:0.843609	validation_1-auc:0.832747
[14]	validation_0-auc:0.844879	validation_1-auc:0.834817
[

[142]	validation_0-auc:0.893627	validation_1-auc:0.842312
[143]	validation_0-auc:0.894016	validation_1-auc:0.842249
[144]	validation_0-auc:0.894087	validation_1-auc:0.842244
[145]	validation_0-auc:0.894171	validation_1-auc:0.842329
[146]	validation_0-auc:0.894259	validation_1-auc:0.842516
[147]	validation_0-auc:0.894301	validation_1-auc:0.84249
[148]	validation_0-auc:0.89468	validation_1-auc:0.842343
[149]	validation_0-auc:0.894979	validation_1-auc:0.842466
[150]	validation_0-auc:0.895049	validation_1-auc:0.84252
[151]	validation_0-auc:0.895147	validation_1-auc:0.842557
[152]	validation_0-auc:0.895426	validation_1-auc:0.842565
[153]	validation_0-auc:0.895661	validation_1-auc:0.842424
[154]	validation_0-auc:0.895794	validation_1-auc:0.842283
[155]	validation_0-auc:0.895887	validation_1-auc:0.842322
[156]	validation_0-auc:0.895989	validation_1-auc:0.842284
[157]	validation_0-auc:0.896027	validation_1-auc:0.842377
[158]	validation_0-auc:0.896145	validation_1-auc:0.84232
[159]	validation_0

XGBClassifier(colsample_bytree=0.75, max_depth=5, n_estimators=1000,
              random_state=156, reg_alpha=0.03)

In [20]:
# AUC 출력
xgb_roc_auc = roc_auc_score(
    y_test
    , xgb_clf.predict_proba(X_test)[:,1]
    , average='macro'
)
print('AUC : {0:.4f}'.format(xgb_roc_auc))

AUC : 0.8453


### LightGBM 사용

In [21]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=1000)

In [23]:
# 검증 데이터 셋 지정
evals= [(X_test, y_test)]
lgbm_clf.fit(
      X_train
    , y_train
    , early_stopping_rounds=200
    , eval_metric='auc'
    , eval_set=evals
    , verbose=True
)
lgbm_auc = roc_auc_score(
    y_test
    , lgbm_clf.predict_proba(X_test)[:,1]
    , average='macro'
)
print('AUC : {0:.4f}'.format(lgbm_auc))

[1]	valid_0's auc: 0.817384	valid_0's binary_logloss: 0.165046
Training until validation scores don't improve for 200 rounds
[2]	valid_0's auc: 0.818903	valid_0's binary_logloss: 0.160006
[3]	valid_0's auc: 0.827707	valid_0's binary_logloss: 0.156323
[4]	valid_0's auc: 0.832155	valid_0's binary_logloss: 0.153463
[5]	valid_0's auc: 0.834677	valid_0's binary_logloss: 0.151256
[6]	valid_0's auc: 0.834093	valid_0's binary_logloss: 0.149427
[7]	valid_0's auc: 0.837046	valid_0's binary_logloss: 0.147961
[8]	valid_0's auc: 0.837838	valid_0's binary_logloss: 0.146591
[9]	valid_0's auc: 0.839435	valid_0's binary_logloss: 0.145455
[10]	valid_0's auc: 0.83973	valid_0's binary_logloss: 0.144486
[11]	valid_0's auc: 0.839799	valid_0's binary_logloss: 0.143769
[12]	valid_0's auc: 0.840034	valid_0's binary_logloss: 0.143146
[13]	valid_0's auc: 0.840271	valid_0's binary_logloss: 0.142533
[14]	valid_0's auc: 0.840342	valid_0's binary_logloss: 0.142036
[15]	valid_0's auc: 0.840928	valid_0's binary_loglos

[132]	valid_0's auc: 0.836005	valid_0's binary_logloss: 0.140827
[133]	valid_0's auc: 0.835753	valid_0's binary_logloss: 0.140876
[134]	valid_0's auc: 0.835732	valid_0's binary_logloss: 0.14087
[135]	valid_0's auc: 0.835679	valid_0's binary_logloss: 0.140901
[136]	valid_0's auc: 0.835577	valid_0's binary_logloss: 0.140947
[137]	valid_0's auc: 0.835408	valid_0's binary_logloss: 0.140986
[138]	valid_0's auc: 0.83532	valid_0's binary_logloss: 0.141039
[139]	valid_0's auc: 0.835139	valid_0's binary_logloss: 0.141118
[140]	valid_0's auc: 0.835142	valid_0's binary_logloss: 0.141143
[141]	valid_0's auc: 0.835196	valid_0's binary_logloss: 0.141144
[142]	valid_0's auc: 0.835175	valid_0's binary_logloss: 0.141154
[143]	valid_0's auc: 0.835202	valid_0's binary_logloss: 0.141159
[144]	valid_0's auc: 0.835281	valid_0's binary_logloss: 0.141168
[145]	valid_0's auc: 0.835105	valid_0's binary_logloss: 0.141234
[146]	valid_0's auc: 0.835241	valid_0's binary_logloss: 0.14124
[147]	valid_0's auc: 0.83516

In [24]:
# LightGBM -> GridSearchCV
import time
from sklearn.model_selection import GridSearchCV

lgbm_clf = LGBMClassifier(n_estimators=200)
params = {
      'num_leaves' : [32,64]
    , 'max_depth' : [128,160]
    , 'min_child_samples' : [60,100]
    , 'subsample' : [0.8,1]
}

In [25]:
gridcv = GridSearchCV(lgbm_clf, param_grid=params)

In [26]:
start_time=time.time()
gridcv.fit(
      X_train
    , y_train
    , early_stopping_rounds=200
    , eval_metric='auc'
    , eval_set=[(X_train, y_train),(X_test, y_test)]
    , verbose=True
)
end_time=time.time()

print('학습시간', end_time-start_time)

[1]	valid_0's auc: 0.823664	valid_0's binary_logloss: 0.156198	valid_1's auc: 0.821078	valid_1's binary_logloss: 0.16488
Training until validation scores don't improve for 200 rounds
[2]	valid_0's auc: 0.82966	valid_0's binary_logloss: 0.150925	valid_1's auc: 0.826326	valid_1's binary_logloss: 0.159774
[3]	valid_0's auc: 0.834498	valid_0's binary_logloss: 0.147158	valid_1's auc: 0.826852	valid_1's binary_logloss: 0.156113
[4]	valid_0's auc: 0.83904	valid_0's binary_logloss: 0.144173	valid_1's auc: 0.831417	valid_1's binary_logloss: 0.153186
[5]	valid_0's auc: 0.841964	valid_0's binary_logloss: 0.141745	valid_1's auc: 0.834593	valid_1's binary_logloss: 0.150797
[6]	valid_0's auc: 0.845253	valid_0's binary_logloss: 0.139712	valid_1's auc: 0.836659	valid_1's binary_logloss: 0.149062
[7]	valid_0's auc: 0.846545	valid_0's binary_logloss: 0.138053	valid_1's auc: 0.836148	valid_1's binary_logloss: 0.147537
[8]	valid_0's auc: 0.848885	valid_0's binary_logloss: 0.1366	valid_1's auc: 0.838153	va

[85]	valid_0's auc: 0.90277	valid_0's binary_logloss: 0.113473	valid_1's auc: 0.832803	valid_1's binary_logloss: 0.140931
[86]	valid_0's auc: 0.903386	valid_0's binary_logloss: 0.113303	valid_1's auc: 0.832806	valid_1's binary_logloss: 0.140945
[87]	valid_0's auc: 0.903657	valid_0's binary_logloss: 0.113153	valid_1's auc: 0.832841	valid_1's binary_logloss: 0.140981
[88]	valid_0's auc: 0.903796	valid_0's binary_logloss: 0.113064	valid_1's auc: 0.832706	valid_1's binary_logloss: 0.141049
[89]	valid_0's auc: 0.904041	valid_0's binary_logloss: 0.112955	valid_1's auc: 0.832649	valid_1's binary_logloss: 0.141079
[90]	valid_0's auc: 0.904277	valid_0's binary_logloss: 0.112805	valid_1's auc: 0.832456	valid_1's binary_logloss: 0.141129
[91]	valid_0's auc: 0.90452	valid_0's binary_logloss: 0.112658	valid_1's auc: 0.832685	valid_1's binary_logloss: 0.141089
[92]	valid_0's auc: 0.904652	valid_0's binary_logloss: 0.112546	valid_1's auc: 0.832534	valid_1's binary_logloss: 0.14115
[93]	valid_0's auc:

[167]	valid_0's auc: 0.918522	valid_0's binary_logloss: 0.104559	valid_1's auc: 0.828216	valid_1's binary_logloss: 0.143527
[168]	valid_0's auc: 0.918607	valid_0's binary_logloss: 0.104472	valid_1's auc: 0.828258	valid_1's binary_logloss: 0.143552
[169]	valid_0's auc: 0.918965	valid_0's binary_logloss: 0.104323	valid_1's auc: 0.828513	valid_1's binary_logloss: 0.14351
[170]	valid_0's auc: 0.919269	valid_0's binary_logloss: 0.104203	valid_1's auc: 0.828485	valid_1's binary_logloss: 0.14352
[171]	valid_0's auc: 0.919326	valid_0's binary_logloss: 0.104122	valid_1's auc: 0.828442	valid_1's binary_logloss: 0.143518
[172]	valid_0's auc: 0.919642	valid_0's binary_logloss: 0.103999	valid_1's auc: 0.828554	valid_1's binary_logloss: 0.143522
[173]	valid_0's auc: 0.91974	valid_0's binary_logloss: 0.103907	valid_1's auc: 0.828517	valid_1's binary_logloss: 0.143538
[174]	valid_0's auc: 0.919949	valid_0's binary_logloss: 0.10381	valid_1's auc: 0.828679	valid_1's binary_logloss: 0.143545
[175]	valid_

[35]	valid_0's auc: 0.880929	valid_0's binary_logloss: 0.122397	valid_1's auc: 0.836618	valid_1's binary_logloss: 0.140034
[36]	valid_0's auc: 0.881746	valid_0's binary_logloss: 0.122132	valid_1's auc: 0.836812	valid_1's binary_logloss: 0.140013
[37]	valid_0's auc: 0.882622	valid_0's binary_logloss: 0.121854	valid_1's auc: 0.836926	valid_1's binary_logloss: 0.139982
[38]	valid_0's auc: 0.883018	valid_0's binary_logloss: 0.121618	valid_1's auc: 0.836863	valid_1's binary_logloss: 0.139968
[39]	valid_0's auc: 0.883857	valid_0's binary_logloss: 0.12136	valid_1's auc: 0.837075	valid_1's binary_logloss: 0.139959
[40]	valid_0's auc: 0.884437	valid_0's binary_logloss: 0.121112	valid_1's auc: 0.836834	valid_1's binary_logloss: 0.139996
[41]	valid_0's auc: 0.88528	valid_0's binary_logloss: 0.120861	valid_1's auc: 0.836028	valid_1's binary_logloss: 0.140135
[42]	valid_0's auc: 0.885917	valid_0's binary_logloss: 0.120634	valid_1's auc: 0.835765	valid_1's binary_logloss: 0.140168
[43]	valid_0's auc

[118]	valid_0's auc: 0.912305	valid_0's binary_logloss: 0.108835	valid_1's auc: 0.835209	valid_1's binary_logloss: 0.141003
[119]	valid_0's auc: 0.912396	valid_0's binary_logloss: 0.10875	valid_1's auc: 0.835087	valid_1's binary_logloss: 0.141043
[120]	valid_0's auc: 0.912515	valid_0's binary_logloss: 0.108654	valid_1's auc: 0.835135	valid_1's binary_logloss: 0.141044
[121]	valid_0's auc: 0.912666	valid_0's binary_logloss: 0.108563	valid_1's auc: 0.834927	valid_1's binary_logloss: 0.141097
[122]	valid_0's auc: 0.912801	valid_0's binary_logloss: 0.108453	valid_1's auc: 0.83459	valid_1's binary_logloss: 0.141163
[123]	valid_0's auc: 0.912941	valid_0's binary_logloss: 0.108335	valid_1's auc: 0.834322	valid_1's binary_logloss: 0.14121
[124]	valid_0's auc: 0.91307	valid_0's binary_logloss: 0.108247	valid_1's auc: 0.834493	valid_1's binary_logloss: 0.141172
[125]	valid_0's auc: 0.913245	valid_0's binary_logloss: 0.108151	valid_1's auc: 0.834464	valid_1's binary_logloss: 0.141201
[126]	valid_

...

[51]	valid_0's auc: 0.911115	valid_0's binary_logloss: 0.110282	valid_1's auc: 0.838459	valid_1's binary_logloss: 0.139669
[52]	valid_0's auc: 0.911601	valid_0's binary_logloss: 0.10999	valid_1's auc: 0.838303	valid_1's binary_logloss: 0.139708
[53]	valid_0's auc: 0.911908	valid_0's binary_logloss: 0.109751	valid_1's auc: 0.838117	valid_1's binary_logloss: 0.13975
[54]	valid_0's auc: 0.912261	valid_0's binary_logloss: 0.109498	valid_1's auc: 0.838252	valid_1's binary_logloss: 0.139711
[55]	valid_0's auc: 0.912616	valid_0's binary_logloss: 0.109226	valid_1's auc: 0.837936	valid_1's binary_logloss: 0.13983
[56]	valid_0's auc: 0.913202	valid_0's binary_logloss: 0.108936	valid_1's auc: 0.83804	valid_1's binary_logloss: 0.13983
[57]	valid_0's auc: 0.913592	valid_0's binary_logloss: 0.10867	valid_1's auc: 0.838083	valid_1's binary_logloss: 0.139818
[58]	valid_0's auc: 0.913818	valid_0's binary_logloss: 0.108421	valid_1's auc: 0.837716	valid_1's binary_logloss: 0.139971
[59]	valid_0's auc: 0.

[117]	valid_0's auc: 0.93053	valid_0's binary_logloss: 0.0976376	valid_1's auc: 0.830085	valid_1's binary_logloss: 0.144358
[118]	valid_0's auc: 0.930553	valid_0's binary_logloss: 0.0975465	valid_1's auc: 0.829901	valid_1's binary_logloss: 0.144437
[119]	valid_0's auc: 0.930549	valid_0's binary_logloss: 0.0974653	valid_1's auc: 0.829668	valid_1's binary_logloss: 0.14454
[120]	valid_0's auc: 0.930978	valid_0's binary_logloss: 0.0972504	valid_1's auc: 0.829504	valid_1's binary_logloss: 0.144619
[121]	valid_0's auc: 0.931184	valid_0's binary_logloss: 0.0971395	valid_1's auc: 0.829359	valid_1's binary_logloss: 0.144679
[122]	valid_0's auc: 0.931477	valid_0's binary_logloss: 0.0969495	valid_1's auc: 0.8293	valid_1's binary_logloss: 0.144754
[123]	valid_0's auc: 0.931535	valid_0's binary_logloss: 0.0968569	valid_1's auc: 0.829075	valid_1's binary_logloss: 0.144847
[124]	valid_0's auc: 0.93182	valid_0's binary_logloss: 0.0966843	valid_1's auc: 0.829082	valid_1's binary_logloss: 0.14487
[125]	

[194]	valid_0's auc: 0.940312	valid_0's binary_logloss: 0.0888382	valid_1's auc: 0.82021	valid_1's binary_logloss: 0.150758
[195]	valid_0's auc: 0.940325	valid_0's binary_logloss: 0.0887805	valid_1's auc: 0.820142	valid_1's binary_logloss: 0.150841
[196]	valid_0's auc: 0.940558	valid_0's binary_logloss: 0.0886109	valid_1's auc: 0.819893	valid_1's binary_logloss: 0.150951
[197]	valid_0's auc: 0.940679	valid_0's binary_logloss: 0.088504	valid_1's auc: 0.819761	valid_1's binary_logloss: 0.151004
[198]	valid_0's auc: 0.940722	valid_0's binary_logloss: 0.088443	valid_1's auc: 0.819583	valid_1's binary_logloss: 0.151085
[199]	valid_0's auc: 0.940848	valid_0's binary_logloss: 0.088333	valid_1's auc: 0.819421	valid_1's binary_logloss: 0.151171
[200]	valid_0's auc: 0.941022	valid_0's binary_logloss: 0.0882187	valid_1's auc: 0.819559	valid_1's binary_logloss: 0.151175
Did not meet early stopping. Best iteration is:
[200]	valid_0's auc: 0.941022	valid_0's binary_logloss: 0.0882187	valid_1's auc: 

[75]	training's auc: 0.910877	training's binary_logloss: 0.111875	valid_1's auc: 0.838923	valid_1's binary_logloss: 0.139443
[76]	training's auc: 0.911102	training's binary_logloss: 0.11173	valid_1's auc: 0.839002	valid_1's binary_logloss: 0.139457
[77]	training's auc: 0.911602	training's binary_logloss: 0.111531	valid_1's auc: 0.839043	valid_1's binary_logloss: 0.139459
[78]	training's auc: 0.911795	training's binary_logloss: 0.111397	valid_1's auc: 0.838884	valid_1's binary_logloss: 0.139477
[79]	training's auc: 0.912087	training's binary_logloss: 0.111234	valid_1's auc: 0.838969	valid_1's binary_logloss: 0.139438
[80]	training's auc: 0.912344	training's binary_logloss: 0.111086	valid_1's auc: 0.838817	valid_1's binary_logloss: 0.139474
[81]	training's auc: 0.912721	training's binary_logloss: 0.110877	valid_1's auc: 0.838967	valid_1's binary_logloss: 0.139485
[82]	training's auc: 0.913095	training's binary_logloss: 0.11068	valid_1's auc: 0.838992	valid_1's binary_logloss: 0.139453
[8

[157]	training's auc: 0.93306	training's binary_logloss: 0.100733	valid_1's auc: 0.836447	valid_1's binary_logloss: 0.14078
[158]	training's auc: 0.93323	training's binary_logloss: 0.100633	valid_1's auc: 0.836448	valid_1's binary_logloss: 0.140799
[159]	training's auc: 0.933337	training's binary_logloss: 0.100518	valid_1's auc: 0.836195	valid_1's binary_logloss: 0.14083
[160]	training's auc: 0.933718	training's binary_logloss: 0.100358	valid_1's auc: 0.836198	valid_1's binary_logloss: 0.140856
[161]	training's auc: 0.934028	training's binary_logloss: 0.100233	valid_1's auc: 0.836221	valid_1's binary_logloss: 0.140863
[162]	training's auc: 0.934239	training's binary_logloss: 0.10009	valid_1's auc: 0.836101	valid_1's binary_logloss: 0.140908
[163]	training's auc: 0.934326	training's binary_logloss: 0.100003	valid_1's auc: 0.836248	valid_1's binary_logloss: 0.14087
[164]	training's auc: 0.934368	training's binary_logloss: 0.0999502	valid_1's auc: 0.836009	valid_1's binary_logloss: 0.1409

In [28]:
# best Parameters
gridcv.best_params_

{'max_depth': 128, 'min_child_samples': 60, 'num_leaves': 32, 'subsample': 0.8}

In [29]:
lgbm_auc = roc_auc_score(
    y_test
    , gridcv.predict_proba(X_test)[:,1]
    , average='macro'
)
print(lgbm_auc)

0.8338552165224338
